# Analyse et nettoyage des données Immatriculation

## Import

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.types import BooleanType, FloatType
from pyspark.sql.functions import trim, lower, col, count, regexp_replace, max, min, when

## Session spark

In [2]:
spark = SparkSession.builder\
    .appName("CleanImmatriculation")\
    .enableHiveSupport()\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/14 16:13:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Import des données

In [3]:
spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

df_immat = spark.sql("SELECT * FROM immatriculations_ext")
df_client = spark.sql("SELECT * FROM clients")

df_immat.show()
df_client.show()
df_client.count()

+---------------+----------+----------------+---------+-----------+--------+--------+-------+--------+-----+
|immatriculation|    marque|             nom|puissance|   longueur|nbplaces|nbportes|couleur|occasion| prix|
+---------------+----------+----------------+---------+-----------+--------+--------+-------+--------+-----+
|     9113 ZK 61|      Saab|        9.3 1.8T|      150|     longue|       5|       5|  blanc|   false|38600|
|     9088 UW 52|    Nissan|   Maxima 3.0 V6|      200|tr�s longue|       5|       5|   bleu|   false|30000|
|     3594 DX 21|Volkswagen|     Polo 1.2 6V|       55|     courte|       5|       3|   gris|    true| 8540|
|     9665 OD 25|   Renault|Vel Satis 3.5 V6|      245|tr�s longue|       5|       5|  rouge|   false|49200|
|     3745 QT 98|      Saab|        9.3 1.8T|      150|     longue|       5|       5|   gris|    true|27020|
|      110 UO 91|    Nissan|      Almera 1.8|      115|    moyenne|       5|       5|   bleu|   false|16450|
|     3024 NS 68|  

100001

## Analyse

### Casse
- Renommage de la colonne "nom" en "modele" dans la table **immatriculation**
- Normalisation des marques de la table **immatriculation**
- Normalisation des modèles de la table **immatriculation**
- Suppression de la 1ère ligne de la table **clients**

#### Traitement Immatriculation

In [4]:
#Renommage de la colonne "nom" en "modele"
df_immat = df_immat.withColumnRenamed("nom", "modele")

df_immat.printSchema()

root
 |-- immatriculation: string (nullable = true)
 |-- marque: string (nullable = true)
 |-- modele: string (nullable = true)
 |-- puissance: integer (nullable = true)
 |-- longueur: string (nullable = true)
 |-- nbplaces: integer (nullable = true)
 |-- nbportes: integer (nullable = true)
 |-- couleur: string (nullable = true)
 |-- occasion: string (nullable = true)
 |-- prix: integer (nullable = true)



In [5]:
#Normalisation des marques
df_immat = df_immat.withColumn("marque", lower(trim(col("marque"))))

#Normalisation des modèles
df_immat = df_immat.withColumn("modele", lower(trim(col("modele"))))

In [6]:
df_immat.show(n=5)

+---------------+----------+----------------+---------+-----------+--------+--------+-------+--------+-----+
|immatriculation|    marque|          modele|puissance|   longueur|nbplaces|nbportes|couleur|occasion| prix|
+---------------+----------+----------------+---------+-----------+--------+--------+-------+--------+-----+
|     9113 ZK 61|      saab|        9.3 1.8t|      150|     longue|       5|       5|  blanc|   false|38600|
|     9088 UW 52|    nissan|   maxima 3.0 v6|      200|tr�s longue|       5|       5|   bleu|   false|30000|
|     3594 DX 21|volkswagen|     polo 1.2 6v|       55|     courte|       5|       3|   gris|    true| 8540|
|     9665 OD 25|   renault|vel satis 3.5 v6|      245|tr�s longue|       5|       5|  rouge|   false|49200|
|     3745 QT 98|      saab|        9.3 1.8t|      150|     longue|       5|       5|   gris|    true|27020|
+---------------+----------+----------------+---------+-----------+--------+--------+-------+--------+-----+
only showing top 5 

#### Traitemment Client

In [7]:
#Suppression de la 1ère ligne de la table clients

df_client = df_client.filter(df_client['immatriculation'] != 'immatriculation')

df_client.show(n=3)

+---+----+----+------------------+---------------+---------------+---------------+
|age|sexe|taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|immatriculation|
+---+----+----+------------------+---------------+---------------+---------------+
| 40|   M| 400|         En Couple|              1|          false|     7982 PU 33|
| 42|   M| 525|         En Couple|              1|          false|     7501 DZ 26|
| 28|   M|1066|       C�libataire|              0|          false|     6461 JH 46|
+---+----+----+------------------+---------------+---------------+---------------+
only showing top 3 rows



### Recherche des caractères spéciaux

- Recherche du symbole "�"
- Recherche du caractère "ã©"

In [8]:
df_client.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sexe: string (nullable = true)
 |-- taux: integer (nullable = true)
 |-- situationfamiliale: string (nullable = true)
 |-- nbenfantacharge: integer (nullable = true)
 |-- deuxiemevoiture: boolean (nullable = true)
 |-- immatriculation: string (nullable = true)



In [9]:
#Recherche du symbole "�"

#table immatriculation
df_search_special_char_immat = df_immat.filter(
    col("immatriculation").like("%�%") | 
    col("marque").like("%�%") | 
    col("modele").like("%�%") | 
    col("longueur").like("%�%") |
    col("couleur").like("%�%") | 
    col("occasion").like("%�%")
)

#table clients
df_search_special_char_client = df_client.filter(
    col("sexe").like("%�%") |
    col("immatriculation").like("%�%")
)

value_client = df_client.groupBy("situationfamiliale").count()


In [10]:
df_search_special_char_immat.show()
df_search_special_char_client.show()
value_client.show()

+---------------+--------+----------------+---------+-----------+--------+--------+-------+--------+-----+
|immatriculation|  marque|          modele|puissance|   longueur|nbplaces|nbportes|couleur|occasion| prix|
+---------------+--------+----------------+---------+-----------+--------+--------+-------+--------+-----+
|     9088 UW 52|  nissan|   maxima 3.0 v6|      200|tr�s longue|       5|       5|   bleu|   false|30000|
|     9665 OD 25| renault|vel satis 3.5 v6|      245|tr�s longue|       5|       5|  rouge|   false|49200|
|     3024 NS 68|   volvo|          s80 t6|      272|tr�s longue|       5|       5|  rouge|   false|50500|
|      985 ZD 20|     bmw|              m5|      507|tr�s longue|       5|       5|  rouge|   false|94800|
|     1988 QH 68| renault|vel satis 3.5 v6|      245|tr�s longue|       5|       5|   noir|   false|49200|
|     7526 DQ 31|     bmw|              m5|      507|tr�s longue|       5|       5|   bleu|   false|94800|
|     8557 XV 50|     bmw|           

+---+----+----+------------------+---------------+---------------+---------------+
|age|sexe|taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|immatriculation|
+---+----+----+------------------+---------------+---------------+---------------+
+---+----+----+------------------+---------------+---------------+---------------+

+------------------+-----+
|situationfamiliale|count|
+------------------+-----+
|          Divorc�e|   56|
|          Mari�(e)|  635|
|              Seul|  298|
|       C�libataire|29862|
|             Seule| 4787|
|               N/D|   94|
|                  |   97|
|         En Couple|64056|
|                 ?|  115|
+------------------+-----+



In [11]:
#Recherche du caractère "ã©"

#table immatriculation
df_search_special_char_2_immat = df_immat.filter(
    col("immatriculation").like("%ã©%") | 
    col("marque").like("%ã©%") | 
    col("modele").like("%ã©%") | 
    col("longueur").like("%ã©%") |
    col("couleur").like("%ã©%") | 
    col("occasion").like("%ã©%")
)

#table clients
df_search_special_char_2_client = df_client.filter(
    col("sexe").like("%ã©%") | 
    col("situationfamiliale").like("%ã©%") | 
    col("immatriculation").like("%ã©%")
)

In [12]:
df_search_special_char_2_immat.show()
df_search_special_char_2_client.show()

+---------------+------+------+---------+--------+--------+--------+-------+--------+----+
|immatriculation|marque|modele|puissance|longueur|nbplaces|nbportes|couleur|occasion|prix|
+---------------+------+------+---------+--------+--------+--------+-------+--------+----+
+---------------+------+------+---------+--------+--------+--------+-------+--------+----+

+---+----+----+------------------+---------------+---------------+---------------+
|age|sexe|taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|immatriculation|
+---+----+----+------------------+---------------+---------------+---------------+
+---+----+----+------------------+---------------+---------------+---------------+



### Correction des caractères spéciaux

- Correction du symbole "�" dans la colonne "longueur".


In [13]:
#Correction du symbole "�" dans la colonne "longueur"

df_immat = df_immat.withColumn("longueur", regexp_replace(col("longueur"), "�", "e"))

df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "�", "e"))


In [14]:

df_search_special_char_immat = df_immat.filter(
    col("immatriculation").like("%�%") | 
    col("marque").like("%�%") | 
    col("modele").like("%�%") | 
    col("longueur").like("%�%") |
    col("couleur").like("%�%") | 
    col("occasion").like("%�%")
)

value_client = df_client.groupBy("situationfamiliale").count()

value_client.show()

df_search_special_char_immat.show()

+------------------+-----+
|situationfamiliale|count|
+------------------+-----+
|              Seul|  298|
|       Celibataire|29862|
|             Seule| 4787|
|          Marie(e)|  635|
|               N/D|   94|
|                  |   97|
|         En Couple|64056|
|                 ?|  115|
|          Divorcee|   56|
+------------------+-----+



+---------------+------+------+---------+--------+--------+--------+-------+--------+----+
|immatriculation|marque|modele|puissance|longueur|nbplaces|nbportes|couleur|occasion|prix|
+---------------+------+------+---------+--------+--------+--------+-------+--------+----+
+---------------+------+------+---------+--------+--------+--------+-------+--------+----+



## Recherche et correction des Null

- Recherche des null

In [15]:

#Recherche des null

# Liste des colonnes du DataFrame
colonnes_immat = df_immat.columns

# Filtrer pour conserver uniquement les lignes où au moins une colonne est nulle
df_immat_nulls = df_immat.filter(
    sum(col(colonne_immat).isNull().cast("int") for colonne_immat in colonnes_immat) > 0
)

# Liste des colonnes du DataFrame
colonnes_client = df_client.columns

# Filtrer pour conserver uniquement les lignes où au moins une colonne est nulle
df_client_nulls = df_client.filter(
    sum(col(colonne_client).isNull().cast("int") for colonne_client in colonnes_client) > 0
)

In [16]:
# Afficher les lignes avec des valeurs nulles
df_immat_nulls.show()
df_client_nulls.show()

+---------------+------+------+---------+--------+--------+--------+-------+--------+----+
|immatriculation|marque|modele|puissance|longueur|nbplaces|nbportes|couleur|occasion|prix|
+---------------+------+------+---------+--------+--------+--------+-------+--------+----+
+---------------+------+------+---------+--------+--------+--------+-------+--------+----+

+----+----+----+------------------+---------------+---------------+---------------+
| age|sexe|taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|immatriculation|
+----+----+----+------------------+---------------+---------------+---------------+
|null|   M|1179|       Celibataire|              0|          false|     4277 VS 97|
|null|   M| 426|         En Couple|              1|          false|     1192 RE 88|
|  37|   M|null|       Celibataire|              0|          false|     8980 MQ 19|
|  26|   M|null|         En Couple|              3|          false|     5356 OE 81|
|  45|   M| 408|         En Couple|           n

## Traitement des nulls dans la table **age**

- Analyse des valeurs inférieur à 18.
- Mettre à null les valeurs inférieures à 18 ans.
- Calculer la médiane.
- Remplacer les nulls par la médiane.

In [17]:
#Analyse des valeurs inférieur à 18.

# Ajouter une colonne qui spécifie si l'âge est inférieur à 18
df_client = df_client.withColumn("age_below_18", when(col("age") < 18, 1).otherwise(0))

# Compter les lignes où "age_below_18" est 1
count_grouped = df_client.groupBy("age_below_18").count()



In [18]:
# Afficher les résultats
count_grouped.show()


+------------+-----+
|age_below_18|count|
+------------+-----+
|           1|  103|
|           0|99897|
+------------+-----+



In [19]:
#Mettre à null les valeurs inférieures à 18 ans.

df_client = df_client.withColumn("age", when(col("age") == -1, None).otherwise(col("age")))
df_client = df_client.withColumn("age", when(col("age") == 0, None).otherwise(col("age")))

#Calculer la médiane.
mediane = df_client.approxQuantile("age", [0.5], 0.01)[0]

#Remplacer les nulls par la médiane.
df_client = df_client.withColumn("age", when(col("age").isNull(), mediane).otherwise(col("age")))




In [20]:
# Afficher la médiane
print("Médiane:", mediane)

value_client = df_client.groupBy("age").count()


value_client.show()

Médiane: 42.0


+----+-----+
| age|count|
+----+-----+
|70.0|  763|
|67.0|  789|
|69.0|  801|
|49.0| 1645|
|29.0| 2464|
|75.0|  795|
|64.0|  869|
|47.0| 1667|
|42.0| 1978|
|44.0| 1604|
|35.0| 1592|
|62.0|  810|
|18.0| 2516|
|80.0|  785|
|39.0| 1700|
|37.0| 1655|
|34.0| 1661|
|25.0| 2461|
|36.0| 1676|
|41.0| 1637|
+----+-----+
only showing top 20 rows



## traitement des null dans **taux**

- Nettoyer situation familiale
    -- Fusionner seul/seule en Celibataire
    -- Remplacer les null/? par N/D

- faire la moyenne des taux par valeur de situation familiale
- Remplacer les nulls

In [21]:
#Nettoyer situation familiale

#Fusionner seul/seule en Celibataire
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "Seule", "Celibataire"))
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "Seul", "Celibataire"))
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "Divorcee", "Divorce"))

df_client = df_client.withColumn("situationfamiliale", replace(col("situationfamiliale"), "N/D", null))
#df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "?", "N/D"))

NameError: name 'replace' is not defined

In [ ]:
value_client = df_client.groupBy("situationfamiliale").count()
value_client.show()

## Modification du type des colonnes occasion

- Recherche des valeurs d'occasion
- Changer le type de la colonne

In [ ]:
#Recherche des valeurs d'occasion
value_count_occasion = df_immat.groupBy("occasion").count()

value_count_occasion.show()

In [ ]:
#Changer le type de la colonne
df_immat = df_immat.withColumn("occasion", df_immat["occasion"].cast(BooleanType()))

df_immat.printSchema()

## Recherche de valeurs aberrantes

- Modéliser les valeurs dans un diagramme à moustaches pour mettre en avant les valeurs aberrantes.
- Afficher les valeurs suspectes.

In [ ]:
#Modéliser les valeurs dans un diagramme à moustaches pour mettre en avant les valeurs aberrantes.

# 2. Sélectionner uniquement les colonnes numériques nécessaires

df_place_porte = df_immat.select(["nbplaces", "nbportes"])
df_prix = df_immat.select(["prix"])
df_puissance = df_immat.select(["puissance"])

# 3. Convertir le DataFrame PySpark en DataFrame Pandas
df_place_porte= df_place_porte.toPandas()
df_prix= df_prix.toPandas()
df_puissance= df_puissance.toPandas()

# 4. Tracer le diagramme à moustaches pour chaque colonne
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_place_porte)
plt.title("Diagramme à moustache pour détecter les valeurs aberrantes places/portes")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 8))
sns.boxplot(data=df_puissance)
plt.title("Diagramme à moustache pour détecter les valeurs aberrantes puissance")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 8))
sns.boxplot(data=df_prix)
plt.title("Diagramme à moustache pour détecter les valeurs aberrantes prix")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Afficher les valeurs suspectes.

# Trouver la valeur maximale de la colonne "prix"
prix_max = df_immat.select(max("prix")).collect()[0][0]

# Filtrer le DataFrame pour afficher les lignes ayant cette valeur
df_max_prix = df_immat.filter(df_immat["prix"] == prix_max)

# Afficher les lignes avec le prix maximum
df_max_prix.show()

# Trouver la valeur maximale de la colonne "prix"
prix_min = df_immat.select(min("prix")).collect()[0][0]

# Filtrer le DataFrame pour afficher les lignes ayant cette valeur
df_min_prix = df_immat.filter(df_immat["prix"] == prix_min)

# Afficher les lignes avec le prix maximum
df_min_prix.show()